In [1]:
import warnings
import logging

root_logger = logging.getLogger()
root_logger.setLevel(logging.ERROR)
warnings.filterwarnings('ignore')

In [2]:
# 复制DataFrame以进行特征工程
df_features = df.copy()

# 1. 时间相关特征
df_features['datetime'] = pd.to_datetime(df_features['datetime'])
df_features['weekday'] = df_features['datetime'].dt.weekday
df_features['is_weekend'] = df_features['weekday'].isin([5, 6]).astype(int)
df_features['day_period'] = pd.cut(df_features['hour'], 
                                 bins=[-1, 5, 11, 16, 20, 24],
                                 labels=['Dawn', 'Morning', 'Afternoon', 'Evening', 'Night'])

# 2. 互动频率特征
daily_interaction_count = df_features.groupby('date').size().reset_index(name='daily_interactions')
df_features = df_features.merge(daily_interaction_count, on='date')

# 3. 情感强度的移动平均
daily_intensity = df_features.groupby('date')['intensity'].agg(['mean', 'std']).reset_index()
daily_intensity.columns = ['date', 'daily_mean_intensity', 'daily_intensity_std']
df_features = df_features.merge(daily_intensity, on='date')

# 4. 互动类型的多样性指标
daily_type_diversity = df_features.groupby('date')['interaction_type'].agg(lambda x: len(set(x))).reset_index()
daily_type_diversity.columns = ['date', 'interaction_type_diversity']
df_features = df_features.merge(daily_type_diversity, on='date')

# 5. 计算增强版情感热力图数据
enhanced_heatmap_data = df_features.groupby(['date', 'hour']).agg({
    'intensity': ['mean', 'count', 'std'],
    'is_weekend': 'first',
    'interaction_type_diversity': 'first'
}).reset_index()

enhanced_heatmap_data.columns = ['date', 'hour', 'mean_intensity', 'interaction_count', 'intensity_std', 'is_weekend', 'type_diversity']
enhanced_heatmap_data['normalized_intensity'] = (enhanced_heatmap_data['mean_intensity'] - enhanced_heatmap_data['mean_intensity'].min()) / \
    (enhanced_heatmap_data['mean_intensity'].max() - enhanced_heatmap_data['mean_intensity'].min()) * 100

# 更新热力图数据结构
enhanced_heatmap_json = {
    'dates': sorted(enhanced_heatmap_data['date'].unique().tolist()),
    'hours': list(range(24)),
    'data': []
}

for _, row in enhanced_heatmap_data.iterrows():
    enhanced_heatmap_json['data'].append({
        'date': row['date'],
        'hour': row['hour'],
        'value': round(row['normalized_intensity'], 2),
        'raw_intensity': round(row['mean_intensity'], 3),
        'interaction_count': int(row['interaction_count']),
        'intensity_std': round(row['intensity_std'], 3) if not pd.isna(row['intensity_std']) else 0,
        'is_weekend': bool(row['is_weekend']),
        'type_diversity': int(row['type_diversity'])
    })

# 更新visualization_data
visualization_data['emotion_heatmap'] = enhanced_heatmap_json
visualization_data['feature_metadata'] = {
    'time_features': ['weekday', 'is_weekend', 'day_period'],
    'intensity_features': ['daily_mean_intensity', 'daily_intensity_std'],
    'interaction_features': ['daily_interactions', 'interaction_type_diversity']
}

# 保存增强后的数据
with open('analytics/visualization_data.json', 'w', encoding='utf-8') as f:
    json.dump(visualization_data, f, ensure_ascii=False, indent=2)

# 显示特征统计信息
print("\n特征统计信息：")
print(df_features[['daily_mean_intensity', 'daily_intensity_std', 'daily_interactions', 
                  'interaction_type_diversity']].describe().round(3))

NameError: name 'df' is not defined

In [3]:
# 导入必要的库并准备数据结构
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import json
import os

# 创建输出目录
os.makedirs('analytics', exist_ok=True)

# 设置随机种子和基础参数
np.random.seed(42)
n_days = 30
base_date = datetime(2024, 1, 1)

# 定义情感互动类型及其权重
interaction_types = ['任务完成', '卡片使用', '积分交易', '留言互动', '愿望达成']
interaction_weights = {
    '任务完成': 0.8,
    '卡片使用': 0.6,
    '积分交易': 0.4,
    '留言互动': 0.7,
    '愿望达成': 1.0
}

# 生成日期序列和模拟数据
dates = [base_date + timedelta(days=i) for i in range(n_days)]
data = []
for date in dates:
    n_interactions = np.random.randint(3, 9)
    for _ in range(n_interactions):
        interaction_type = np.random.choice(interaction_types)
        hour = np.random.randint(8, 23)
        base_intensity = np.random.random() * 0.5 + 0.5
        intensity = base_intensity * interaction_weights[interaction_type]
        
        data.append({
            'datetime': date.replace(hour=hour).strftime('%Y-%m-%d %H:%M:%S'),
            'date': date.strftime('%Y-%m-%d'),
            'hour': hour,
            'interaction_type': interaction_type,
            'intensity': round(intensity, 3)
        })

# 创建基础DataFrame
df = pd.DataFrame(data)

# 开始特征工程
df_features = df.copy()

# 1. 时间相关特征
df_features['datetime'] = pd.to_datetime(df_features['datetime'])
df_features['weekday'] = df_features['datetime'].dt.weekday
df_features['is_weekend'] = df_features['weekday'].isin([5, 6]).astype(int)
df_features['day_period'] = pd.cut(df_features['hour'], 
                                 bins=[-1, 5, 11, 16, 20, 24],
                                 labels=['Dawn', 'Morning', 'Afternoon', 'Evening', 'Night'])

# 2. 互动频率特征
daily_interaction_count = df_features.groupby('date').size().reset_index(name='daily_interactions')
df_features = df_features.merge(daily_interaction_count, on='date')

# 3. 情感强度的移动平均
daily_intensity = df_features.groupby('date')['intensity'].agg(['mean', 'std']).reset_index()
daily_intensity.columns = ['date', 'daily_mean_intensity', 'daily_intensity_std']
df_features = df_features.merge(daily_intensity, on='date')

# 4. 互动类型的多样性指标
daily_type_diversity = df_features.groupby('date')['interaction_type'].agg(lambda x: len(set(x))).reset_index()
daily_type_diversity.columns = ['date', 'interaction_type_diversity']
df_features = df_features.merge(daily_type_diversity, on='date')

# 5. 计算增强版情感热力图数据
enhanced_heatmap_data = df_features.groupby(['date', 'hour']).agg({
    'intensity': ['mean', 'count', 'std'],
    'is_weekend': 'first',
    'interaction_type_diversity': 'first'
}).reset_index()

enhanced_heatmap_data.columns = ['date', 'hour', 'mean_intensity', 'interaction_count', 'intensity_std', 'is_weekend', 'type_diversity']
enhanced_heatmap_data['normalized_intensity'] = (enhanced_heatmap_data['mean_intensity'] - enhanced_heatmap_data['mean_intensity'].min()) / \
    (enhanced_heatmap_data['mean_intensity'].max() - enhanced_heatmap_data['mean_intensity'].min()) * 100

# 创建热力图数据结构
enhanced_heatmap_json = {
    'dates': sorted(enhanced_heatmap_data['date'].unique().tolist()),
    'hours': list(range(24)),
    'data': []
}

for _, row in enhanced_heatmap_data.iterrows():
    enhanced_heatmap_json['data'].append({
        'date': row['date'],
        'hour': row['hour'],
        'value': round(row['normalized_intensity'], 2),
        'raw_intensity': round(row['mean_intensity'], 3),
        'interaction_count': int(row['interaction_count']),
        'intensity_std': round(row['intensity_std'], 3) if not pd.isna(row['intensity_std']) else 0,
        'is_weekend': bool(row['is_weekend']),
        'type_diversity': int(row['type_diversity'])
    })

# 创建并保存完整的可视化数据
visualization_data = {
    'emotion_heatmap': enhanced_heatmap_json,
    'feature_metadata': {
        'time_features': ['weekday', 'is_weekend', 'day_period'],
        'intensity_features': ['daily_mean_intensity', 'daily_intensity_std'],
        'interaction_features': ['daily_interactions', 'interaction_type_diversity']
    }
}

# 保存数据
with open('analytics/visualization_data.json', 'w', encoding='utf-8') as f:
    json.dump(visualization_data, f, ensure_ascii=False, indent=2)

# 显示特征统计信息
print("\n特征统计信息：")
print(df_features[['daily_mean_intensity', 'daily_intensity_std', 'daily_interactions', 
                  'interaction_type_diversity']].describe().round(3))


特征统计信息：
       daily_mean_intensity  daily_intensity_std  daily_interactions  \
count               167.000              167.000             167.000   
mean                  0.510                0.178               6.198   
std                   0.079                0.073               1.740   
min                   0.352                0.021               3.000   
25%                   0.466                0.129               5.000   
50%                   0.494                0.194               7.000   
75%                   0.577                0.222               8.000   
max                   0.838                0.362               8.000   

       interaction_type_diversity  
count                     167.000  
mean                        3.377  
std                         0.804  
min                         1.000  
25%                         3.000  
50%                         3.000  
75%                         4.000  
max                         5.000  


In [4]:
# Print basic info about the main DataFrames
print("Original DataFrame (df) columns:")
print(df.columns.tolist())
print("\nEnhanced DataFrame (df_features) columns:")
print(df_features.columns.tolist())

# Print sample data statistics
print("\nShape of df:", df.shape)
print("Shape of df_features:", df_features.shape)

# Print unique values for categorical columns
print("\nUnique interaction types:", df['interaction_type'].unique())
print("Date range:", df['date'].min(), "to", df['date'].max())
print("Hour range:", df['hour'].min(), "to", df['hour'].max())

# Print interaction weights reference
print("\nInteraction weights:")
for k, v in interaction_weights.items():
    print(f"{k}: {v}")

Original DataFrame (df) columns:
['datetime', 'date', 'hour', 'interaction_type', 'intensity']

Enhanced DataFrame (df_features) columns:
['datetime', 'date', 'hour', 'interaction_type', 'intensity', 'weekday', 'is_weekend', 'day_period', 'daily_interactions', 'daily_mean_intensity', 'daily_intensity_std', 'interaction_type_diversity']

Shape of df: (167, 5)
Shape of df_features: (167, 12)

Unique interaction types: ['愿望达成' '积分交易' '留言互动' '任务完成' '卡片使用']
Date range: 2024-01-01 to 2024-01-30
Hour range: 8 to 22

Interaction weights:
任务完成: 0.8
卡片使用: 0.6
积分交易: 0.4
留言互动: 0.7
愿望达成: 1.0


In [5]:
# Convert datetime to proper format and create time-based features
df_points['datetime'] = pd.to_datetime(df_points['datetime'])
df_points_features = df_points.copy()

# Add time-based features
df_points_features['hour'] = df_points_features['datetime'].dt.hour
df_points_features['weekday'] = df_points_features['datetime'].dt.weekday
df_points_features['is_weekend'] = df_points_features['weekday'].isin([5, 6]).astype(int)
df_points_features['week_number'] = df_points_features['datetime'].dt.isocalendar().week

# Calculate cumulative metrics
df_points_features['cumulative_points'] = df_points_features.groupby(
    df_points_features['datetime'].dt.date)['points'].cumsum()

# Analyze patterns by time periods
time_patterns = df_points_features.groupby('hour').agg({
    'points': ['count', 'mean', 'sum'],
    'type': lambda x: x.value_counts().index[0]  # Most common activity type per hour
}).round(2)

# Calculate activity type trends
activity_trends = df_points_features.groupby(['date', 'activity']).agg({
    'points': ['count', 'sum'],
    'balance': 'last'
}).round(2)

# Generate enhanced time series data
time_series_data = {
    'dates': daily_stats['date'].tolist(),
    'daily_summary': daily_stats.to_dict('records'),
    'hourly_patterns': {
        'hours': list(range(8, 24)),
        'patterns': time_patterns.to_dict()
    },
    'activity_metrics': {
        'by_type': df_points_features.groupby('activity').agg({
            'points': ['count', 'sum', 'mean', 'std'],
            'balance': ['min', 'max']
        }).round(2).to_dict(),
        'by_date_type': activity_trends.to_dict()
    },
    'trend_indicators': {
        'daily_growth_rate': daily_stats['growth_rate'].tolist(),
        'momentum': daily_stats['momentum'].tolist(),
        'balance_trend': daily_stats['end_balance'].tolist()
    }
}

# Update visualization data with enhanced time series information
visualization_data['points_analysis'] = {
    'time_series': time_series_data,
    'metadata': {
        'total_transactions': len(df_points_features),
        'total_points_earned': df_points_features[df_points_features['points'] > 0]['points'].sum(),
        'total_points_spent': abs(df_points_features[df_points_features['points'] < 0]['points'].sum()),
        'average_daily_transactions': daily_stats['transaction_count'].mean(),
        'most_active_hours': time_patterns.index[time_patterns[('points', 'count')].nlargest(3).index].tolist()
    }
}

# Save updated visualization data
with open('analytics/visualization_data.json', 'w', encoding='utf-8') as f:
    json.dump(visualization_data, f, ensure_ascii=False, indent=2)

# Display analysis summary
print("\n=== Points Analysis Summary ===")
print("\nMost Active Hours:")
print(time_patterns[('points', 'count')].nlargest(3))
print("\nActivity Type Distribution:")
print(df_points_features['activity'].value_counts(normalize=True).round(3))
print("\nPoints Balance Trend:")
print(daily_stats[['date', 'end_balance', 'growth_rate']].tail())

NameError: name 'df_points' is not defined

In [6]:
# Print key DataFrames' information
print("df columns:")
print(df.columns)
print("\ndf_features columns:")
print(df_features.columns)
print("\nenhanced_heatmap_data columns:")
print(enhanced_heatmap_data.columns)

print("\nKey variables:")
print("\ninteraction_types:")
print(interaction_types)
print("\ninteraction_weights:")
print(interaction_weights)

print("\nData ranges:")
print(f"Date range: {min(dates)} to {max(dates)}")
print(f"Hour range: {df['hour'].min()} to {df['hour'].max()}")
print(f"Intensity range: {df['intensity'].min():.3f} to {df['intensity'].max():.3f}")

print("\nFeature columns types:")
print(df_features.dtypes)

df columns:
Index(['datetime', 'date', 'hour', 'interaction_type', 'intensity'], dtype='object')

df_features columns:
Index(['datetime', 'date', 'hour', 'interaction_type', 'intensity', 'weekday',
       'is_weekend', 'day_period', 'daily_interactions',
       'daily_mean_intensity', 'daily_intensity_std',
       'interaction_type_diversity'],
      dtype='object')

enhanced_heatmap_data columns:
Index(['date', 'hour', 'mean_intensity', 'interaction_count', 'intensity_std',
       'is_weekend', 'type_diversity', 'normalized_intensity'],
      dtype='object')

Key variables:

interaction_types:
['任务完成', '卡片使用', '积分交易', '留言互动', '愿望达成']

interaction_weights:
{'任务完成': 0.8, '卡片使用': 0.6, '积分交易': 0.4, '留言互动': 0.7, '愿望达成': 1.0}

Data ranges:
Date range: 2024-01-01 00:00:00 to 2024-01-30 00:00:00
Hour range: 8 to 22
Intensity range: 0.203 to 0.992

Feature columns types:
datetime                      datetime64[ns]
date                                  object
hour                              

In [7]:
# Generate points data
point_data = []
balance = 100  # Initial balance

for date in dates:
    n_activities = np.random.randint(2, 7)
    for _ in range(n_activities):
        activity = np.random.choice(list(point_activities.keys()))
        activity_info = point_activities[activity]
        points = activity_info['base_points'] + np.random.randint(-activity_info['variance'], activity_info['variance'] + 1)
        
        activity_time = date.replace(
            hour=np.random.randint(8, 23),
            minute=np.random.randint(0, 60)
        )
        
        balance += points
        point_data.append({
            'datetime': activity_time.strftime('%Y-%m-%d %H:%M:%S'),
            'date': activity_time.strftime('%Y-%m-%d'),
            'activity': activity,
            'type': activity_info['type'],
            'points': points,
            'balance': balance
        })

# Convert to DataFrame and add features
df_points = pd.DataFrame(point_data)
df_points['datetime'] = pd.to_datetime(df_points['datetime'])

# Calculate daily statistics
daily_stats = df_points.groupby('date').agg({
    'points': ['count', 'sum', 'mean', 'std'],
    'balance': 'last'
}).round(2)

daily_stats.columns = ['transaction_count', 'net_points', 'avg_points', 'points_std', 'end_balance']
daily_stats = daily_stats.reset_index()

# Calculate growth metrics
daily_stats['growth_rate'] = daily_stats['net_points'] / daily_stats['transaction_count']
daily_stats['momentum'] = daily_stats['net_points'].rolling(window=3, min_periods=1).mean()

# Generate time series data
time_series_data = {
    'dates': daily_stats['date'].tolist(),
    'daily_metrics': {
        'transactions': daily_stats['transaction_count'].tolist(),
        'net_points': daily_stats['net_points'].tolist(),
        'balance': daily_stats['end_balance'].tolist(),
        'growth_rate': daily_stats['growth_rate'].round(3).tolist(),
        'momentum': daily_stats['momentum'].round(3).tolist()
    },
    'activity_summary': df_points.groupby('activity').agg({
        'points': ['count', 'sum', 'mean'],
        'balance': ['min', 'max']
    }).round(2).to_dict()
}

# Update visualization data
with open('analytics/visualization_data.json', 'r', encoding='utf-8') as f:
    visualization_data = json.load(f)

visualization_data['points_analysis'] = {
    'time_series': time_series_data,
    'metadata': {
        'total_transactions': len(df_points),
        'total_points_earned': df_points[df_points['points'] > 0]['points'].sum(),
        'total_points_spent': abs(df_points[df_points['points'] < 0]['points'].sum()),
        'average_daily_points': daily_stats['net_points'].mean(),
        'balance_range': {
            'min': df_points['balance'].min(),
            'max': df_points['balance'].max()
        }
    }
}

# Save updated data
with open('analytics/visualization_data.json', 'w', encoding='utf-8') as f:
    json.dump(visualization_data, f, ensure_ascii=False, indent=2)

# Display summary statistics
print("\n=== Points Analysis Summary ===")
print("\nDaily Statistics:")
print(daily_stats[['date', 'transaction_count', 'net_points', 'end_balance']].head())
print("\nActivity Distribution:")
print(df_points['activity'].value_counts(normalize=True).round(3))
print("\nPoints Balance Range:")
print(f"Min: {df_points['balance'].min():.2f}, Max: {df_points['balance'].max():.2f}")

NameError: name 'point_activities' is not defined

In [8]:
# Define point values for each interaction type based on existing intensity data
point_weights = {
    '任务完成': 10,
    '卡片使用': 8,
    '积分交易': 5,
    '留言互动': 6,
    '愿望达成': 15
}

# Create points DataFrame from existing df_features
df_points = df_features.copy()

# Calculate points based on interaction type and intensity
df_points['points'] = df_points.apply(lambda row: 
    round(point_weights[row['interaction_type']] * row['intensity']), axis=1)

# Calculate running balance
df_points['balance'] = df_points['points'].cumsum() + 100  # Starting with 100 points

# Calculate daily statistics
daily_stats = df_points.groupby('date').agg({
    'points': ['count', 'sum', 'mean', 'std'],
    'balance': 'last'
}).round(2)

daily_stats.columns = ['transaction_count', 'net_points', 'avg_points', 'points_std', 'end_balance']
daily_stats = daily_stats.reset_index()

# Calculate growth metrics
daily_stats['growth_rate'] = daily_stats['net_points'] / daily_stats['transaction_count']
daily_stats['momentum'] = daily_stats['net_points'].rolling(window=3, min_periods=1).mean()

# Generate time series data
time_series_data = {
    'dates': daily_stats['date'].tolist(),
    'daily_metrics': {
        'transactions': daily_stats['transaction_count'].tolist(),
        'net_points': daily_stats['net_points'].tolist(),
        'balance': daily_stats['end_balance'].tolist(),
        'growth_rate': daily_stats['growth_rate'].round(3).tolist(),
        'momentum': daily_stats['momentum'].round(3).tolist()
    },
    'interaction_summary': df_points.groupby('interaction_type').agg({
        'points': ['count', 'sum', 'mean'],
        'balance': ['min', 'max']
    }).round(2).to_dict()
}

# Update visualization data
with open('analytics/visualization_data.json', 'r', encoding='utf-8') as f:
    visualization_data = json.load(f)

visualization_data['points_analysis'] = {
    'time_series': time_series_data,
    'metadata': {
        'total_interactions': len(df_points),
        'total_points_earned': df_points['points'].sum(),
        'average_points_per_interaction': df_points['points'].mean(),
        'average_daily_points': daily_stats['net_points'].mean(),
        'balance_range': {
            'min': df_points['balance'].min(),
            'max': df_points['balance'].max()
        },
        'most_active_periods': {
            'hours': df_points.groupby('hour')['points'].count().nlargest(3).index.tolist(),
            'weekdays': df_points.groupby('weekday')['points'].count().nlargest(3).index.tolist()
        }
    }
}

# Save updated data
with open('analytics/visualization_data.json', 'w', encoding='utf-8') as f:
    json.dump(visualization_data, f, ensure_ascii=False, indent=2)

# Display summary statistics
print('\n=== Points Analysis Summary ===')
print('\nDaily Statistics:')
print(daily_stats[['date', 'transaction_count', 'net_points', 'end_balance']].head())
print('\nInteraction Type Distribution:')
print(df_points['interaction_type'].value_counts(normalize=True).round(3))
print('\nPoints Balance Range:')
print(f"Min: {df_points['balance'].min():.2f}, Max: {df_points['balance'].max():.2f}")

TypeError: keys must be str, int, float, bool or None, not tuple

In [9]:
# Define point values for each interaction type based on existing intensity data
point_weights = {
    '任务完成': 10,
    '卡片使用': 8,
    '积分交易': 5,
    '留言互动': 6,
    '愿望达成': 15
}

# Create points DataFrame from existing df_features
df_points = df_features.copy()

# Calculate points based on interaction type and intensity
df_points['points'] = df_points.apply(lambda row: 
    round(point_weights[row['interaction_type']] * row['intensity']), axis=1)

# Calculate running balance
df_points['balance'] = df_points['points'].cumsum() + 100  # Starting with 100 points

# Calculate daily statistics with flattened column names
daily_stats_raw = df_points.groupby('date').agg({
    'points': ['count', 'sum', 'mean', 'std'],
    'balance': 'last'
})

# Flatten column names
daily_stats_raw.columns = ['transaction_count', 'net_points', 'avg_points', 'points_std', 'end_balance']
daily_stats = daily_stats_raw.reset_index()

# Calculate growth metrics
daily_stats['growth_rate'] = daily_stats['net_points'] / daily_stats['transaction_count']
daily_stats['momentum'] = daily_stats['net_points'].rolling(window=3, min_periods=1).mean()

# Calculate interaction type summary with flattened structure
interaction_summary = {}
for interaction_type in df_points['interaction_type'].unique():
    stats = df_points[df_points['interaction_type'] == interaction_type].agg({
        'points': ['count', 'sum', 'mean'],
        'balance': ['min', 'max']
    })
    interaction_summary[interaction_type] = {
        'count': int(stats['points']['count']),
        'total_points': float(stats['points']['sum']),
        'avg_points': float(stats['points']['mean']),
        'min_balance': float(stats['balance']['min']),
        'max_balance': float(stats['balance']['max'])
    }

# Generate time series data
time_series_data = {
    'dates': daily_stats['date'].tolist(),
    'daily_metrics': {
        'transactions': daily_stats['transaction_count'].tolist(),
        'net_points': daily_stats['net_points'].tolist(),
        'balance': daily_stats['end_balance'].tolist(),
        'growth_rate': daily_stats['growth_rate'].round(3).tolist(),
        'momentum': daily_stats['momentum'].round(3).tolist()
    },
    'interaction_summary': interaction_summary
}

# Update visualization data
with open('analytics/visualization_data.json', 'r', encoding='utf-8') as f:
    visualization_data = json.load(f)

# Calculate time-based activity patterns
hourly_activity = df_points.groupby('hour')['points'].agg([
    ('count', 'count'),
    ('total_points', 'sum'),
    ('avg_points', 'mean')
]).round(2).to_dict('index')

weekday_activity = df_points.groupby('weekday')['points'].agg([
    ('count', 'count'),
    ('total_points', 'sum'),
    ('avg_points', 'mean')
]).round(2).to_dict('index')

# Add points analysis to visualization data
visualization_data['points_analysis'] = {
    'time_series': time_series_data,
    'metadata': {
        'total_interactions': len(df_points),
        'total_points_earned': float(df_points['points'].sum()),
        'average_points_per_interaction': float(df_points['points'].mean()),
        'average_daily_points': float(daily_stats['net_points'].mean()),
        'balance_range': {
            'min': float(df_points['balance'].min()),
            'max': float(df_points['balance'].max())
        },
        'activity_patterns': {
            'hourly': hourly_activity,
            'weekly': weekday_activity
        }
    }
}

# Save updated data
with open('analytics/visualization_data.json', 'w', encoding='utf-8') as f:
    json.dump(visualization_data, f, ensure_ascii=False, indent=2)

# Display summary statistics
print('\n=== Points Analysis Summary ===')
print('\nDaily Statistics:')
print(daily_stats[['date', 'transaction_count', 'net_points', 'end_balance']].head())
print('\nInteraction Type Distribution:')
print(df_points['interaction_type'].value_counts(normalize=True).round(3))
print('\nPoints Balance Range:')
print(f"Min: {df_points['balance'].min():.2f}, Max: {df_points['balance'].max():.2f}")

JSONDecodeError: Expecting property name enclosed in double quotes: line 1656 column 9 (char 37584)

In [10]:
# Check main dataframes structure
print("Original DataFrame (df) columns:")
print(df.columns.tolist())
print("\nEnhanced DataFrame (df_features) columns:")
print(df_features.columns.tolist())

# Check interaction types and their weights
print("\nInteraction Types and Weights:")
print(interaction_weights)

# Check visualization_data structure
print("\nVisualization Data Structure:")
print("- Dates range:", visualization_data['emotion_heatmap']['dates'][0], "to", visualization_data['emotion_heatmap']['dates'][-1])
print("- Hours range:", min(visualization_data['emotion_heatmap']['hours']), "to", max(visualization_data['emotion_heatmap']['hours']))
print("- Features used:", visualization_data['feature_metadata'])

# Check example of enhanced heatmap data point
print("\nExample Heatmap Data Point:")
print(visualization_data['emotion_heatmap']['data'][0])

Original DataFrame (df) columns:
['datetime', 'date', 'hour', 'interaction_type', 'intensity']

Enhanced DataFrame (df_features) columns:
['datetime', 'date', 'hour', 'interaction_type', 'intensity', 'weekday', 'is_weekend', 'day_period', 'daily_interactions', 'daily_mean_intensity', 'daily_intensity_std', 'interaction_type_diversity']

Interaction Types and Weights:
{'任务完成': 0.8, '卡片使用': 0.6, '积分交易': 0.4, '留言互动': 0.7, '愿望达成': 1.0}

Visualization Data Structure:
- Dates range: 2024-01-01 to 2024-01-30
- Hours range: 0 to 23
- Features used: {'time_features': ['weekday', 'is_weekend', 'day_period'], 'intensity_features': ['daily_mean_intensity', 'daily_intensity_std'], 'interaction_features': ['daily_interactions', 'interaction_type_diversity']}

Example Heatmap Data Point:
{'date': '2024-01-01', 'hour': 11, 'value': 48.67, 'raw_intensity': 0.571, 'interaction_count': 1, 'intensity_std': 0, 'is_weekend': False, 'type_diversity': 3}


In [11]:
# 分析积分获取和消耗的时间模式
df_points_analysis = df_points.copy()

# 添加时间特征
df_points_analysis['hour_bin'] = pd.cut(df_points_analysis['hour'], 
                                      bins=[0, 6, 12, 18, 24],
                                      labels=['Dawn', 'Morning', 'Afternoon', 'Night'])

# 计算各时段的积分统计
time_pattern_stats = df_points_analysis.groupby('hour_bin').agg({
    'points': ['count', 'sum', 'mean', 'std']
}).round(2)
time_pattern_stats.columns = ['transaction_count', 'total_points', 'avg_points', 'points_std']

# 计算每种互动类型的积分贡献
type_contribution = df_points_analysis.groupby('interaction_type').agg({
    'points': ['count', 'sum', 'mean']
}).round(2)
type_contribution.columns = ['count', 'total_points', 'avg_points']
type_contribution['contribution_ratio'] = (type_contribution['total_points'] / type_contribution['total_points'].sum()).round(3)

# 计算积分变化趋势指标
trend_analysis = daily_stats.copy()
trend_analysis['rolling_avg'] = trend_analysis['net_points'].rolling(window=7, min_periods=1).mean()
trend_analysis['volatility'] = trend_analysis['net_points'].rolling(window=7, min_periods=1).std()
trend_analysis['trend'] = trend_analysis['rolling_avg'].diff().apply(lambda x: 'Up' if x > 0 else 'Down' if x < 0 else 'Stable')

# 读取现有的visualization_data
with open('analytics/visualization_data.json', 'r', encoding='utf-8') as f:
    visualization_data = json.load(f)

# 更新积分分析数据
points_analysis_update = {
    'time_patterns': {
        'hourly_distribution': {
            period: {
                'count': int(time_pattern_stats.loc[period, 'transaction_count']),
                'total_points': float(time_pattern_stats.loc[period, 'total_points']),
                'avg_points': float(time_pattern_stats.loc[period, 'avg_points'])
            } for period in time_pattern_stats.index
        },
        'type_contribution': {
            itype: {
                'count': int(type_contribution.loc[itype, 'count']),
                'total_points': float(type_contribution.loc[itype, 'total_points']),
                'contribution_ratio': float(type_contribution.loc[itype, 'contribution_ratio'])
            } for itype in type_contribution.index
        }
    },
    'trend_analysis': {
        'dates': trend_analysis['date'].tolist(),
        'metrics': {
            'rolling_avg': trend_analysis['rolling_avg'].round(2).tolist(),
            'volatility': trend_analysis['volatility'].round(2).tolist(),
            'trend_direction': trend_analysis['trend'].tolist()
        }
    }
}

# 更新visualization_data
visualization_data['points_analysis'].update(points_analysis_update)

# 保存更新后的数据
with open('analytics/visualization_data.json', 'w', encoding='utf-8') as f:
    json.dump(visualization_data, f, ensure_ascii=False, indent=2)

# 显示分析结果
print("\n=== 积分时间模式分析 ===")
print("\n时段分布：")
print(time_pattern_stats)
print("\n互动类型贡献：")
print(type_contribution[['count', 'contribution_ratio']])
print("\n趋势分析样本：")
print(trend_analysis[['date', 'rolling_avg', 'volatility', 'trend']].head())

JSONDecodeError: Expecting property name enclosed in double quotes: line 1656 column 9 (char 37584)

In [12]:
# 分析积分获取和消耗的时间模式
df_points_analysis = df_points.copy()

# 添加时间特征
df_points_analysis['hour_bin'] = pd.cut(df_points_analysis['hour'], 
                                      bins=[0, 6, 12, 18, 24],
                                      labels=['Dawn', 'Morning', 'Afternoon', 'Night'])

# 计算各时段的积分统计
time_pattern_stats = df_points_analysis.groupby('hour_bin').agg({
    'points': ['count', 'sum', 'mean', 'std']
}).round(2)
time_pattern_stats.columns = ['transaction_count', 'total_points', 'avg_points', 'points_std']

# 计算每种互动类型的积分贡献
type_contribution = df_points_analysis.groupby('interaction_type').agg({
    'points': ['count', 'sum', 'mean']
}).round(2)
type_contribution.columns = ['count', 'total_points', 'avg_points']
type_contribution['contribution_ratio'] = (type_contribution['total_points'] / type_contribution['total_points'].sum()).round(3)

# 计算积分变化趋势指标
trend_analysis = daily_stats.copy()
trend_analysis['rolling_avg'] = trend_analysis['net_points'].rolling(window=7, min_periods=1).mean()
trend_analysis['volatility'] = trend_analysis['net_points'].rolling(window=7, min_periods=1).std()
trend_analysis['trend'] = trend_analysis['rolling_avg'].diff().apply(lambda x: 'Up' if x > 0 else 'Down' if x < 0 else 'Stable')

# 创建完整的visualization数据结构
visualization_data = {
    'emotion_heatmap': enhanced_heatmap_json,  # 使用之前生成的热力图数据
    'points_analysis': {
        'time_patterns': {
            'hourly_distribution': {
                period: {
                    'count': int(time_pattern_stats.loc[period, 'transaction_count']),
                    'total_points': float(time_pattern_stats.loc[period, 'total_points']),
                    'avg_points': float(time_pattern_stats.loc[period, 'avg_points'])
                } for period in time_pattern_stats.index
            },
            'type_contribution': {
                itype: {
                    'count': int(type_contribution.loc[itype, 'count']),
                    'total_points': float(type_contribution.loc[itype, 'total_points']),
                    'contribution_ratio': float(type_contribution.loc[itype, 'contribution_ratio'])
                } for itype in type_contribution.index
            }
        },
        'trend_analysis': {
            'dates': trend_analysis['date'].tolist(),
            'metrics': {
                'rolling_avg': trend_analysis['rolling_avg'].round(2).tolist(),
                'volatility': trend_analysis['volatility'].round(2).tolist(),
                'trend_direction': trend_analysis['trend'].tolist()
            }
        },
        'summary_stats': {
            'total_points': float(df_points_analysis['points'].sum()),
            'average_daily_points': float(daily_stats['net_points'].mean()),
            'min_balance': float(df_points_analysis['balance'].min()),
            'max_balance': float(df_points_analysis['balance'].max())
        }
    },
    'metadata': {
        'date_range': {
            'start': df_points_analysis['date'].min(),
            'end': df_points_analysis['date'].max()
        },
        'interaction_types': list(df_points_analysis['interaction_type'].unique())
    }
}

# 保存完整的数据结构
with open('analytics/visualization_data.json', 'w', encoding='utf-8') as f:
    json.dump(visualization_data, f, ensure_ascii=False, indent=2)

# 显示分析结果
print('\n=== 积分时间模式分析 ===')
print('\n时段分布：')
print(time_pattern_stats)
print('\n互动类型贡献：')
print(type_contribution[['count', 'contribution_ratio']])
print('\n趋势分析样本：')
print(trend_analysis[['date', 'rolling_avg', 'volatility', 'trend']].head())


=== 积分时间模式分析 ===

时段分布：
           transaction_count  total_points  avg_points  points_std
hour_bin                                                          
Dawn                       0             0         NaN         NaN
Morning                   63           318        5.05        3.69
Afternoon                 61           289        4.74        3.51
Night                     43           216        5.02        3.76

互动类型贡献：
                  count  contribution_ratio
interaction_type                           
任务完成                 33               0.255
卡片使用                 33               0.142
愿望达成                 31               0.423
留言互动                 32               0.117
积分交易                 38               0.063

趋势分析样本：
         date  rolling_avg  volatility   trend
0  2024-01-01        36.00         NaN  Stable
1  2024-01-02        33.50    3.535534    Down
2  2024-01-03        30.00    6.557439    Down
3  2024-01-04        33.75    9.215024      Up
4  2024-01-0

In [13]:
print("=== df_features columns ===")
print(df_features.columns.tolist())
print("\n=== df_points_analysis columns ===")
print(df_points_analysis.columns.tolist())
print("\n=== trend_analysis columns ===")
print(trend_analysis.columns.tolist())

# Important data patterns and structures
print("\n=== Interaction Types ===")
print(interaction_types)
print("\n=== Interaction Weights ===")
print(interaction_weights)

# Time-related information
print("\n=== Date Range ===")
print(f"Start date: {base_date}")
print(f"Number of days: {n_days}")

# Since we have visualization_data as a complex JSON structure, let's see its keys
print("\n=== Visualization Data Structure ===")
print("Top level keys:", list(visualization_data.keys()))

=== df_features columns ===
['datetime', 'date', 'hour', 'interaction_type', 'intensity', 'weekday', 'is_weekend', 'day_period', 'daily_interactions', 'daily_mean_intensity', 'daily_intensity_std', 'interaction_type_diversity']

=== df_points_analysis columns ===
['datetime', 'date', 'hour', 'interaction_type', 'intensity', 'weekday', 'is_weekend', 'day_period', 'daily_interactions', 'daily_mean_intensity', 'daily_intensity_std', 'interaction_type_diversity', 'points', 'balance', 'hour_bin']

=== trend_analysis columns ===
['date', 'transaction_count', 'net_points', 'avg_points', 'points_std', 'end_balance', 'growth_rate', 'momentum', 'rolling_avg', 'volatility', 'trend']

=== Interaction Types ===
['任务完成', '卡片使用', '积分交易', '留言互动', '愿望达成']

=== Interaction Weights ===
{'任务完成': 0.8, '卡片使用': 0.6, '积分交易': 0.4, '留言互动': 0.7, '愿望达成': 1.0}

=== Date Range ===
Start date: 2024-01-01 00:00:00
Number of days: 30

=== Visualization Data Structure ===
Top level keys: ['emotion_heatmap', 'points_ana

In [14]:
# Import required libraries for ML
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

# Prepare features for ML model
def prepare_ml_features(df):
    # Calculate rolling features
    daily_stats = df.groupby('date').agg({
        'intensity': ['mean', 'std', 'count'],
        'points': ['sum', 'mean'],
        'balance': 'last',
        'interaction_type_diversity': 'first'
    }).reset_index()
    
    daily_stats.columns = ['date', 'mean_intensity', 'std_intensity', 'interaction_count', 
                          'daily_points', 'avg_points', 'balance', 'type_diversity']
    
    # Add rolling features
    for window in [3, 7]:
        daily_stats[f'rolling_{window}d_points'] = daily_stats['daily_points'].rolling(window=window, min_periods=1).mean()
        daily_stats[f'rolling_{window}d_intensity'] = daily_stats['mean_intensity'].rolling(window=window, min_periods=1).mean()
        daily_stats[f'rolling_{window}d_diversity'] = daily_stats['type_diversity'].rolling(window=window, min_periods=1).mean()
    
    return daily_stats

# Calculate ML-based growth score
def calculate_ml_growth_score(df):
    # Prepare features
    daily_stats = prepare_ml_features(df)
    
    # Create target variable (simplified growth score based on multiple factors)
    daily_stats['base_score'] = (
        (daily_stats['interaction_count'] / daily_stats['interaction_count'].max()) * 0.3 +
        (daily_stats['balance'] / daily_stats['balance'].max()) * 0.2 +
        (daily_stats['type_diversity'] / daily_stats['type_diversity'].max()) * 0.3 +
        (1 - daily_stats['std_intensity'] / daily_stats['std_intensity'].max()) * 0.2
    )
    
    # Prepare features for ML model
    feature_cols = ['mean_intensity', 'std_intensity', 'interaction_count', 'daily_points', 
                   'avg_points', 'type_diversity', 'rolling_3d_points', 'rolling_7d_points',
                   'rolling_3d_intensity', 'rolling_7d_intensity', 'rolling_3d_diversity',
                   'rolling_7d_diversity']
    
    X = daily_stats[feature_cols]
    y = daily_stats['base_score']
    
    # Scale features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Train ML models
    rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
    xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
    
    rf_model.fit(X_scaled, y)
    xgb_model.fit(X_scaled, y)
    
    # Combine predictions
    rf_pred = rf_model.predict(X_scaled)
    xgb_pred = xgb_model.predict(X_scaled)
    ensemble_pred = (rf_pred + xgb_pred) / 2
    
    # Create final growth scores DataFrame
    growth_scores = pd.DataFrame({
        'date': daily_stats['date'],
        'ml_score': ensemble_pred,
        'interaction_score': daily_stats['interaction_count'] / daily_stats['interaction_count'].max(),
        'points_score': daily_stats['balance'] / daily_stats['balance'].max(),
        'diversity_score': daily_stats['type_diversity'] / daily_stats['type_diversity'].max(),
        'stability_score': 1 - daily_stats['std_intensity'] / daily_stats['std_intensity'].max()
    })
    
    # Calculate total score (combining ML and rule-based scores)
    growth_scores['total_score'] = growth_scores['ml_score'].clip(0, 1)
    
    return growth_scores

# Calculate growth scores using ML approach
ml_growth_scores = calculate_ml_growth_score(df_points_analysis)

# Define and identify milestones
def identify_ml_milestones(growth_scores):
    milestones = []
    score_thresholds = [0.3, 0.5, 0.7, 0.9]
    milestone_names = ['初步了解', '稳定互动', '深入发展', '紧密关系']
    
    for threshold, name in zip(score_thresholds, milestone_names):
        scores_above = growth_scores[growth_scores['total_score'] >= threshold]
        if not scores_above.empty:
            milestone_date = scores_above.iloc[0]['date']
            milestones.append({
                'name': name,
                'date': milestone_date,
                'score': float(scores_above.iloc[0]['total_score']),
                'features': {
                    'interaction': float(scores_above.iloc[0]['interaction_score']),
                    'points': float(scores_above.iloc[0]['points_score']),
                    'diversity': float(scores_above.iloc[0]['diversity_score']),
                    'stability': float(scores_above.iloc[0]['stability_score'])
                }
            })
    
    return milestones

# Get milestones
ml_milestones = identify_ml_milestones(ml_growth_scores)

# Prepare growth analysis data
ml_growth_analysis = {
    'growth_curve': {
        'dates': ml_growth_scores['date'].tolist(),
        'scores': {
            'total': ml_growth_scores['total_score'].round(3).tolist(),
            'interaction': ml_growth_scores['interaction_score'].round(3).tolist(),
            'points': ml_growth_scores['points_score'].round(3).tolist(),
            'diversity': ml_growth_scores['diversity_score'].round(3).tolist(),
            'stability': ml_growth_scores['stability_score'].round(3).tolist()
        }
    },
    'milestones': ml_milestones,
    'current_progress': {
        'score': float(ml_growth_scores['total_score'].iloc[-1]),
        'level': next((m['name'] for m in reversed(ml_milestones) 
                      if m['score'] <= ml_growth_scores['total_score'].iloc[-1]), '初始阶段')
    },
    'model_metrics': {
        'feature_importance': {
            'interaction': 0.3,
            'points': 0.2,
            'diversity': 0.3,
            'stability': 0.2
        }
    }
}

# Update visualization data
visualization_data['relationship_growth'] = ml_growth_analysis

# Save updated visualization data
with open('analytics/visualization_data.json', 'w', encoding='utf-8') as f:
    json.dump(visualization_data, f, ensure_ascii=False, indent=2)

# Display analysis results
print('\n=== ML-Based Relationship Growth Analysis ===')
print('\nGrowth Curve Sample (First 5 days):')
print(ml_growth_scores[['date', 'total_score']].head())
print('\nMilestones Achieved:')
for milestone in ml_milestones:
    print(f"{milestone['name']}: {milestone['date']} (Score: {milestone['score']:.3f})")
print(f"\nCurrent Progress: {ml_growth_analysis['current_progress']['level']} "
      f"(Score: {ml_growth_analysis['current_progress']['score']:.3f})")


=== ML-Based Relationship Growth Analysis ===

Growth Curve Sample (First 5 days):
         date  total_score
0  2024-01-01     0.528481
1  2024-01-02     0.608766
2  2024-01-03     0.667583
3  2024-01-04     0.716699
4  2024-01-05     0.425860

Milestones Achieved:
初步了解: 2024-01-01 (Score: 0.528)
稳定互动: 2024-01-01 (Score: 0.528)
深入发展: 2024-01-04 (Score: 0.717)

Current Progress: 深入发展 (Score: 0.811)


In [15]:
import json
from datetime import datetime

# Read existing visualization_data.json
with open('analytics/visualization_data.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Update metadata with comprehensive information
metadata = {
    'date_range': {
        'start': '2024-01-01',
        'end': '2024-01-30'
    },
    'data_types': [
        'emotion_heatmap',
        'points_analysis',
        'relationship_growth'
    ],
    'interaction_types': ['任务完成', '卡片使用', '积分交易', '留言互动', '愿望达成'],
    'version': '1.0',
    'metrics': {
        'emotion_intensity': {
            'min': 0,
            'max': 1,
            'weights': interaction_weights
        },
        'growth_factors': {
            'interaction': 0.3,
            'points': 0.2,
            'diversity': 0.3,
            'stability': 0.2
        }
    }
}

# Update data structure with metadata
data['metadata'] = metadata
data['last_updated'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

# Verify data structure completeness
required_sections = ['emotion_heatmap', 'points_analysis', 'relationship_growth']
for section in required_sections:
    if section not in data:
        print(f'警告：缺少 {section} 部分')

# Add data quality metrics
data['data_quality'] = {
    'total_records': len(df),
    'date_coverage': f"{len(df['date'].unique())} days",
    'completeness': {
        'emotion_data': len(df[~df['intensity'].isna()]) / len(df),
        'points_data': len(df_points_analysis[~df_points_analysis['points'].isna()]) / len(df_points_analysis),
        'growth_data': len(ml_growth_scores[~ml_growth_scores['total_score'].isna()]) / len(ml_growth_scores)
    }
}

# Save the final visualization data
with open('analytics/visualization_data.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

# Display data structure overview
print('\n=== 可视化数据结构概览 ===')
for key, value in data.items():
    if isinstance(value, dict):
        print(f'\n{key}:')
        for subkey in value.keys():
            print(f'  - {subkey}')
    else:
        print(f'\n{key}: {value}')

print('\n数据质量指标：')
print(json.dumps(data['data_quality'], ensure_ascii=False, indent=2))
print('\n文件已保存到: analytics/visualization_data.json')


=== 可视化数据结构概览 ===

emotion_heatmap:
  - dates
  - hours
  - data

points_analysis:
  - time_patterns
  - trend_analysis
  - summary_stats

metadata:
  - date_range
  - data_types
  - interaction_types
  - version
  - metrics

relationship_growth:
  - growth_curve
  - milestones
  - current_progress
  - model_metrics

last_updated: 2025-02-20 17:52:57

data_quality:
  - total_records
  - date_coverage
  - completeness

数据质量指标：
{
  "total_records": 167,
  "date_coverage": "30 days",
  "completeness": {
    "emotion_data": 1.0,
    "points_data": 1.0,
    "growth_data": 1.0
  }
}

文件已保存到: analytics/visualization_data.json
